In [30]:
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
# https://medium.com/@kurtsenol21/topic-modeling-lda-mallet-implementation-in-python-part-1-c493a5297ad2
# https://jeriwieringa.com/projects/dissertation/
# https://radimrehurek.com/gensim/models/wrappers/ldamallet.html

# Packages

import os
import pandas as pd
import json
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime as dt
import re
from itertools import compress
import ast
import spacy
import nltk
import gensim

nlp = spacy.load("da_core_news_sm", disable=['parser', 'ner'])

data_path = os.path.join('D:/', 'data', 'horesta')
out_path = os.path.join('..', 'output')

In [31]:
# Loading data
path = os.path.join(data_path, 'horesta_posts_2021-02-02.json')

with open(path, 'r') as file:
    data = json.load(file)
    
len(data)

1261

In [32]:
# Converting data to data frame

df = pd.DataFrame.from_records(data)

,url,accessed,title,tags,links,publish_date,access_date,text,html
0,https://horesta.dk/nyheder/2020/december/det-e...,1,Det er tid til at få det lange lys på,"[coronakrise, horesta, kirsten munch, vaccinat...",[],04-12 - 2020,2020-12-04,\n\n\n\n\n\n\n\nFra næste måned begynder udrul...,"\n<!DOCTYPE html>\n\n<html lang=""da"">\n<head>\..."
1,https://horesta.dk/nyheder/2020/december/forsl...,1,Forslag om lavere moms,"[small danish hotels, jørgen christensen, moms...",[],04-12 - 2020,2020-12-04,\n\n\n\n\n\n\n\nDer er akut behov for at få st...,"\n<!DOCTYPE html>\n\n<html lang=""da"">\n<head>\..."
2,https://horesta.dk/nyheder/2020/december/hores...,1,HORESTA: Feriepenge bør følges op med et oplev...,"[feriepenge, hjælpepakker, turisme, dansk turi...",[],03-12 - 2020,2020-12-04,"\n\n\n\n\n\n\n\nHORESTA tager positivt imod, a...","\n<!DOCTYPE html>\n\n<html lang=""da"">\n<head>\..."
3,https://horesta.dk/nyheder/2020/december/webin...,1,Webinar med Danske Bank: Her bruger danskerne ...,"[webinar, forbrug, danske bank, louise aggerst...","[/webinar-med-danske-bank, /cdn-cgi/l/email-pr...",03-12 - 2020,2020-12-04,\n\n\n\n\n\n\n\nDanskernes forbrugsvaner har æ...,"\n<!DOCTYPE html>\n\n<html lang=""da"">\n<head>\..."
4,https://horesta.dk/nyheder/2020/december/sidst...,1,Sidste chance for finansiering – hør mere på w...,"[vækstfonden, finansiering, coronakrise]",[/events/2020/december/webinar-med-vaekstfonde...,02-12 - 2020,2020-12-04,\n\n\n\n\n\n\n\nHORESTA inviterer til webinar ...,"\n<!DOCTYPE html>\n\n<html lang=""da"">\n<head>\..."


In [27]:
# TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

texts = list(df['text'])

vectorizer = TfidfVectorizer()
tfidf_m = vectorizer.fit_transform(texts)

features = vectorizer.get_feature_names()

tfidf_df = pd.DataFrame(vectorizer.idf_, index = features)
tfidf_df = tfidf_df.sort_values(0, ascending = True).reset_index()
tfidf_df.head(50) # Under 1.8 virker som en god grænse

,index,0
0,og,1.006359
1,til,1.024059
2,at,1.028941
3,for,1.028941
4,på,1.029757
5,er,1.035488
6,af,1.063799
7,med,1.084278
8,det,1.089464
9,en,1.094676


In [28]:
posts = list(df['text'])

In [47]:
posts[:1]

['\n\n\n\n\n\n\n\nFra næste måned begynder udrulningen af det statslige vaccinationsprogram.Dermed er der håb om, at den sidste bjergetape er kørt, og vi kan tage hul på de lange og flade strækninger i retning af en genåbning og normalisering af vores samfund.Ruten vil stadig være uforudsigelig og formentlig ujævn og bumlet. Men retningen er den rigtige, og dermed skaber det nye muligheder for at begynde at planlægge, hvordan og hvornår vores erhverv kan begynde at skabe omsætning og indtjening igen. Netop sådan en genåbningsplan efterlyste vis, da regeringen lancerede sin vaccinationsplan. En plan, som kan gøre det muligt for vores erhverv at tænde for det lange lys og begynde at planlægge to, tre og fire måneder frem.Det er oplagt at udnytte de muligheder, vaccinationer i kombination med hurtigtests kan skabe for at lempe på de nuværende restriktioner. Og det bør være muligt i løbet af første halvår af 2021 at lave store events og arrangementer uden begrænsninger på antal eller kvadr

In [48]:
def texts_to_words(texts):
    for text in texts:
        yield(gensim.utils.simple_preprocess(str(text), deacc=True))  # deacc=True removes punctuations

data_words = list(texts_to_words(posts))

print(data_words[:1])

[['fra', 'næste', 'maned', 'begynder', 'udrulningen', 'af', 'det', 'statslige', 'dermed', 'er', 'der', 'hab', 'om', 'at', 'den', 'sidste', 'bjergetape', 'er', 'kørt', 'og', 'vi', 'kan', 'tage', 'hul', 'pa', 'de', 'lange', 'og', 'flade', 'strækninger', 'retning', 'af', 'en', 'genabning', 'og', 'normalisering', 'af', 'vores', 'samfund', 'ruten', 'vil', 'stadig', 'være', 'uforudsigelig', 'og', 'formentlig', 'ujævn', 'og', 'bumlet', 'men', 'retningen', 'er', 'den', 'rigtige', 'og', 'dermed', 'skaber', 'det', 'nye', 'muligheder', 'for', 'at', 'begynde', 'at', 'planlægge', 'hvordan', 'og', 'hvornar', 'vores', 'erhverv', 'kan', 'begynde', 'at', 'skabe', 'omsætning', 'og', 'indtjening', 'igen', 'netop', 'sadan', 'en', 'genabningsplan', 'efterlyste', 'vis', 'da', 'regeringen', 'lancerede', 'sin', 'en', 'plan', 'som', 'kan', 'gøre', 'det', 'muligt', 'for', 'vores', 'erhverv', 'at', 'tænde', 'for', 'det', 'lange', 'lys', 'og', 'begynde', 'at', 'planlægge', 'to', 'tre', 'og', 'fire', 'maneder', 'f

In [63]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['fra', 'næste', 'maned', 'begynder', 'udrulningen', 'af', 'det', 'statslige', 'dermed', 'er', 'der', 'hab', 'om', 'at', 'den', 'sidste', 'bjergetape', 'er', 'kørt', 'og', 'vi', 'kan', 'tage', 'hul', 'pa', 'de', 'lange', 'og', 'flade', 'strækninger', 'retning', 'af', 'en', 'genabning', 'og', 'normalisering', 'af', 'vores', 'samfund', 'ruten', 'vil', 'stadig', 'være', 'uforudsigelig', 'og', 'formentlig', 'ujævn', 'og', 'bumlet', 'men', 'retningen', 'er', 'den', 'rigtige', 'og', 'dermed', 'skaber', 'det', 'nye', 'muligheder', 'for', 'at', 'begynde', 'at', 'planlægge', 'hvordan', 'og', 'hvornar', 'vores', 'erhverv', 'kan', 'begynde', 'at', 'skabe', 'omsætning', 'og', 'indtjening', 'igen', 'netop', 'sadan', 'en', 'genabningsplan', 'efterlyste', 'vis', 'da', 'regeringen', 'lancerede', 'sin', 'en', 'plan', 'som', 'kan', 'gøre', 'det', 'muligt', 'for', 'vores', 'erhverv', 'at', 'tænde', 'for', 'det', 'lange', 'lys', 'og', 'begynde', 'at', 'planlægge', 'to', 'tre', 'og', 'fire', 'maneder', 'fr

In [24]:
# stopwords

stop_words = nltk.corpus.stopwords.words('danish')
stop_words = list(set(stop_words + list(tfidf_df.loc[tfidf_df[0] < 1.85, 'index'])))

In [ ]:
# Tokenizer

def tokenizer_custom(text, stop_words=stop_words, tags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    doc = nlp(text)
        
    pos_tags = tags
    
    tokens = []
      
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.pos in pos_tags:
                token = word.text.lower() # Returning the word in lower-case.
                tokens.append(token)
    
    return(tokens)